# Explore Exercises

These exercises should go in a notebook or script named `explore`.

Answer the questions below by using a combination of the techniques discussed in the lesson that you think is appropriate.

In [1]:
import warnings

warnings.filterwarnings("ignore")

import pyspark.sql
from pyspark.sql.functions import *

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/15 14:59:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/15 14:59:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


- How many different cases are there, by department?


- Does the percentage of cases that are late vary by department?


- On average, how late are the late cases by department?


- What is the service type that is the most late? Just for Parks & Rec?


- For the DSD/Code Enforcement department, what are the most common service request types? Look at other departments too.


- Does whether or not its a weekend matter for when a case is opened/closed?


- On average, how many cases are opened a day for the Customer Service department?


- Does the number of service requests for the solid waste department vary by day of the week?